<a href="https://www.kaggle.com/code/rahulchauhan016/deepspace-navigator-ai?scriptVersionId=296205818" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# ✅  1 — Import Libraries

In [37]:
import pandas as pd
import numpy as np

from IPython.display import display, Image
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# ✅  2 — Load Dataset

In [3]:
DATA_PATH = "/kaggle/input/nasa-astronomy-picture-of-the-day-1995-2026/nasa_apod_complete.csv"

df = pd.read_csv(DATA_PATH)

print("Rows:", len(df))
df.head()


Rows: 11186


,date,explanation,hdurl,media_type,service_version,title,url,copyright
0,1995-06-16,Today's Picture: Explanation: If the Earth...,https://apod.nasa.gov/apod/image/e_lens.gif,image,v1,Neutron Star Earth,https://apod.nasa.gov/apod/image/e_lens.gif,NaN
1,1995-06-20,"Today's Picture: June 20, 1995 The Pleiades...",https://apod.nasa.gov/apod/image/pleiades2.gif,image,v1,Pleiades Star Cluster,https://apod.nasa.gov/apod/image/pleiades2.gif,NaN
2,1995-06-21,"Today's Picture: June 21, 1995 The Aftermat...",https://apod.nasa.gov/apod/image/sn1987a_hst.gif,image,v1,Supernova 1987a Aftermath,https://apod.nasa.gov/apod/image/sn1987a_hst.gif,NaN
3,1995-06-22,In 1972 Astronauts on the United States's last...,https://apod.nasa.gov/apod/image/earth_a17.gif,image,v1,Earth from Apollo 17,https://apod.nasa.gov/apod/image/earth_a17.gif,NaN
4,1995-06-23,"What if you could ""see"" gamma rays? This compu...",https://apod.nasa.gov/apod/image/egret_gro.gif,image,v1,Gamma Ray Sky Map,https://apod.nasa.gov/apod/image/egret_gro.gif,NaN


# ✅  3 — Clean Dataset

In [47]:
# lowercase column names
df.columns = df.columns.str.lower()

# convert date
df["date"] = pd.to_datetime(df["date"])

# keep only images
df = df[df["media_type"] == "image"]

# remove missing explanations
df = df.dropna(subset=["explanation"])

# remove duplicates
df = df.drop_duplicates(subset="title")

# fill missing titles
df["title"] = df["title"].fillna("Unknown")

# create searchable text
df["search_text"] = df["title"] + " " + df["explanation"]

print("Clean rows:", len(df))


Clean rows: 7732


# ✅  4 — Build Smart Search Engine

In [48]:
vectorizer = TfidfVectorizer(
    stop_words="english",
    max_features=5000
)

tfidf_matrix = vectorizer.fit_transform(df["search_text"])

print("Search engine ready!")



Search engine ready!


# ✅  5 — Find Best Match Function

In [49]:
def find_best_answer(question, df):

    q_vec = vectorizer.transform([question])

    similarity = cosine_similarity(q_vec, tfidf_matrix)

    best_idx = similarity[0].argsort()[::-1][0]

    return df.iloc[best_idx]


# ✅  6 — Short Explanation Function

In [50]:
def short_answer(text, sentences=3):
    parts = text.split(".")
    return ". ".join(parts[:sentences]) + "."


# ✅  7 — Assistant Response Function

In [51]:
def assistant_reply(question):

    result = find_best_answer(question, df)

    print("\n🚀 Space Assistant")
    print("Topic:", result["title"])

    display(Image(url=result["url"]))

    print("\nExplanation:")
    print(short_answer(result["explanation"]))

    print("\n" + "="*60 + "\n")


# ✅ 8 — Chat Assistant Loop

In [52]:
def space_chat():
    print("🌌 Space Assistant Ready!")
    print("Type 'exit' to stop.\n")

    while True:
        q = input("You: ")

        if q.lower() == "exit":
            print("Assistant: Goodbye astronaut!")
            break

        assistant_reply(q)


# ✅ 9 — Start Chat

In [53]:
space_chat()


🌌 Space Assistant Ready!
Type 'exit' to stop.



You:  why is mars red



🚀 Space Assistant
Topic: Mars at Closest Approach



Explanation:
When does Mars appear the largest? This occurs when Earth sweeps past Mars in their respective orbits around the Sun, creating a momentary Sun-Earth-Mars alignment called opposition.  The featured image shows the Mars opposition that occurred earlier this year, as well as how Mars will look later this year.   Mars actually changes its size continuously -- the monthly jumps in size are illustrative.




You:  what is a nebula


...

You:  how do black holes form



🚀 Space Assistant
Topic: GW190521: Unexpected Black Holes Collide



Explanation:
How do black holes like this form? The two black holes that spiraled together to produce the gravitational wave event GW190521 were not only the most massive black holes ever seen by LIGO and VIRGO so far, their masses -- 66 and 85 solar masses -- were unprecedented and unexpected.  Lower mass black holes, below about 65 solar masses are known to form in supernova explosions.   Conversely, higher mass black holes, above about 135 solar masses, are thought to be created by very massive stars imploding after they use up their weight-bearing nuclear-fusion-producing elements.




You:  exit


Assistant: Goodbye astronaut!


# ✅  10 — Chat Memory

In [54]:
chat_memory = []


# ✅  11 — Context Question Builder

In [58]:
def build_context_question(question):
    
    if len(chat_memory) > 0 and len(question.split()) < 5:
        last_topic = chat_memory[-1]
        return last_topic + " " + question
    
    return question


# ✅  12 — Update Assistant Reply

In [59]:
def assistant_reply(question):

    contextual_question = build_context_question(question)

    result = find_best_answer(contextual_question, df)

    # save memory
    chat_memory.append(question)

    print("\n🚀 Space Assistant")
    print("Topic:", result["title"])

    display(Image(url=result["url"]))

    print("\nExplanation:")
    print(short_answer(result["explanation"]))

    print("\n" + "="*60 + "\n")


# ✅ 13 — Chat Loop With Memory Reset

In [60]:
def space_chat():
    global chat_memory
    
    print("🌌 Space Assistant Ready!")
    print("Type 'exit' to stop.")
    print("Type 'clear' to reset memory.\n")

    while True:
        q = input("You: ")

        if q.lower() == "exit":
            print("Assistant: Goodbye astronaut!")
            break

        if q.lower() == "clear":
            chat_memory = []
            print("Memory cleared.")
            continue

        assistant_reply(q)


# ✅ 14 — Start Chat

In [62]:
space_chat()


🌌 Space Assistant Ready!
Type 'exit' to stop.
Type 'clear' to reset memory.



You:  tell me about mars



🚀 Space Assistant
Topic: Mars at Closest Approach



Explanation:
When does Mars appear the largest? This occurs when Earth sweeps past Mars in their respective orbits around the Sun, creating a momentary Sun-Earth-Mars alignment called opposition.  The featured image shows the Mars opposition that occurred earlier this year, as well as how Mars will look later this year.   Mars actually changes its size continuously -- the monthly jumps in size are illustrative.




You:  why is it red?


...

You:  stop



🚀 Space Assistant
Topic: Io Rotating



Explanation:
 More careful inspection does verify continuous volcanic plumes.inglileo




You:  exit


Assistant: Goodbye astronaut!
